In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging

from fundus_data_toolkit.datamodules import CLASSIF_PATHS, SEG_PATHS, USER_SETTING, Task, register_paths
from fundus_data_toolkit.datamodules.classification import (
    AptosDataModule,
    DDRDataModule,
    EyePACSDataModule,
    IDRiDDataModule,
)
from fundus_data_toolkit.datamodules.utils import merge_existing_datamodules
from nntools import NNOpt

root = logging.getLogger()
root.setLevel(logging.NOTSET)


In [6]:
paths = {
'EYEPACS': '/home/clement/Documents/data/eyepacs/',
'APTOS': '/home/clement/Documents/data/aptos/',
'DDR': '/home/clement/Documents/data/DDR-dataset/DR_grading/',
'IDRID': '/home/clement/Documents/data/IDRID/B. Disease Grading/', 
}
register_paths(paths, task=Task.CLASSIFICATION)

In [7]:
CLASSIF_PATHS

{'EYEPACS': '/home/clement/Documents/data/eyepacs/',
 'APTOS': '/home/clement/Documents/data/aptos/',
 'DDR': '/home/clement/Documents/data/DDR-dataset/DR_grading/',
 'IDRID': '/home/clement/Documents/data/IDRID/B. Disease Grading/'}

In [10]:
img_size = (512, 512)
idrid_datamodule = IDRiDDataModule(CLASSIF_PATHS.IDRID, img_size=img_size, batch_size=64, 
                                   valid_size=0.2,
                                   use_cache=True, num_workers=8,
                                   precise_autocrop=True,
                                   cache_option=NNOpt.CACHE_MEMORY).setup_all()
eyepacs_datamodule = EyePACSDataModule(CLASSIF_PATHS.EYEPACS, img_size=img_size, batch_size=64, use_cache=True, num_workers=8,
                                       precise_autocrop=True,
                                   cache_option=NNOpt.CACHE_MEMORY).setup_all()
aptos_datamodule = AptosDataModule(CLASSIF_PATHS.APTOS, img_size=img_size, batch_size=64, use_cache=True, num_workers=8,
                                   precise_autocrop=True,
                                   cache_option=NNOpt.CACHE_MEMORY).setup_all()
ddr_datamodule = DDRDataModule(CLASSIF_PATHS.DDR, img_size=img_size, batch_size=64, use_cache=True, num_workers=8,
                                   cache_option=NNOpt.CACHE_MEMORY, precise_autocrop=True).setup_all()
datamodules = {"IDRID": idrid_datamodule, "EYEPACS": eyepacs_datamodule, "APTOS": aptos_datamodule, "DDR": ddr_datamodule}
# ddr_datamodule.train.plot(0)

In [19]:
for k, v, in datamodules.items():
    total_images = len(v.train) + (len(v.val) if v.val is not None else 0) + (len(v.test) if v.test is not None else 0)
    print(f"{k} Total: {total_images} (tra12i1
          ``````````n: {total_images - (len(v.test) if v.test is not None else 0)})")
    if v.val is None and v.test is None:x#
        print(f"Train: {len(v.train)}")
    elif v.val is not None and v.test is not None:
        print(f"Train: {len(v.train)}, Test: {len(v.test)}, Valid: {len(v.val)}")
    elif v.val is not None:
        print(f"Train: {len(v.train)}, Valid: {len(v.val)}")
    else:
        print(f"Train: {len(v.train)}, Test: {len(v.test)}")


IDRID Total: 516 (train: 413)
Train: 331, Test: 103, Valid: 82
EYEPACS Total: 88702 (train: 35126)
Train: 35126, Test: 53576
APTOS Total: 3662 (train: 3662)
Train: 3662
DDR Total: 12522 (train: 8763)
Train: 6260, Test: 3759, Valid: 2503
